In [24]:
filename='2003-2023'
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from pathlib import Path
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
# Veri işleme ve Algoritma Performans süresi için kullanılan modüller
from sklearn.preprocessing import LabelEncoder
import datetime
from time import time
import lightgbm as lgb #LightGBM
import warnings
warnings.filterwarnings("ignore")
df = pd.read_csv(filename+".csv", delimiter=',')
print("Orjinal Veri")
print(df.shape)
print(df.head())
yDegiskeni = []
haftaninGunu = []
hangiAy = []
ayBasi = []
aySonu = []
target=[]
for i in range(0,len(df)):
    tTarih=datetime.datetime.strptime(df['Date'][i], '%Y-%m-%d') # datetime.datetime(2021, 7, 31, 0, 0) formatı gibi yazılmasını sağlar
    hGunu = tTarih.strftime("%A")
    hAy = tTarih.strftime("%B")
    hAyRakam = tTarih.strftime("%m")
    hAyGunu = tTarih.strftime("%d")
    hYil = tTarih.strftime("%Y")
    intAyGunu = int(hAyGunu)
    intAy = int(hAyRakam)
    intYil = int(hYil)
    nTarget=0
    if float(df['High'][i])<float(df['Low'][i]):nTarget = 0
    elif float(df['High'][i])>float(df['Low'][i]):nTarget = 1
    else:pass
    if intAyGunu == 1:
        hAybasi = "yes" 
        hAysonu = "no"
    elif (intAy==4 or intAy==6 or intAy==9 or intAy==11) and intAyGunu == 30:
        hAybasi = "no"
        hAysonu = "yes"
    elif 29 < intAyGunu <= 31 and (intAy==1 or intAy==3 or intAy==5 or intAy==7 or intAy==8 or intAy==10 or intAy==12):
        hAybasi = "no"
        hAysonu = "yes"
        if intAyGunu == 30:
            hAybasi = "no"
            hAysonu = "no"
    elif 1 < intAyGunu < 30 and intAy!=2:
        hAybasi = "no"
        hAysonu = "no"
    elif intAy==2:
        if ((intYil%400==0) or (intYil%100!=0)and (intYil%4==0)):
            hAybasi = "no"
            hAysonu = "no"
            if intAyGunu == 29:
                hAybasi = "no"
                hAysonu = "yes"
        else :
            hAybasi = "no"
            hAysonu = "no"
            if intAyGunu == 28:
                hAybasi = "no"
                hAysonu = "yes"
    else:
        hAybasi = ""
        hAysonu = ""
    target.append(nTarget)
    haftaninGunu.append(hGunu) 
    hangiAy.append(hAy)
    ayBasi.append(hAybasi) 
    aySonu.append(hAysonu)
    yDegiskeni.append("no")
len(yDegiskeni)
df['y'] = yDegiskeni
df['Haftanın Günü'] = haftaninGunu
df['Ay'] = hangiAy
df['Ay Başı'] = ayBasi
df['Ay Sonu'] = aySonu
df['H-L'] = target
y= (df['y'] == 'yes')*1
df.drop('y', axis=1, inplace = True)


#df.prcp.fillna(0 ,inplace = True)

df.columns = ['Tarih','Açılış','Yüksek','Düşük','Kapanış','Adj_Kapanış','Hacim','Haftanın Günü','Ay','Ay Başı','Ay Sonu','H-L']
df.isnull()
df.Düşük.fillna(0 ,inplace = True)
df.Açılış.fillna(0 ,inplace = True)
df.Yüksek.fillna(0 ,inplace = True)
df.Kapanış.fillna(0 ,inplace = True)
df.Adj_Kapanış.fillna(0 ,inplace = True)
df.Hacim.fillna(0 ,inplace = True)
print(df.isnull().sum())
#df.to_csv('out.csv')
islemSutunlari = ['Haftanın Günü','Ay','Ay Başı','Ay Sonu','H-L']

for i in islemSutunlari:
    le = LabelEncoder()
    df[i] = le.fit_transform(df[i])
print(df.head())
filename = filename + "-İşlenmiş"
print(filename)
df.to_csv(filename+'.csv')




Orjinal Veri
(5158, 7)
         Date          Open          High           Low         Close  \
0  2003-01-02  10369.900391  10642.299805  10247.400391  10598.599609   
1  2003-01-03  10598.599609  10837.500000  10549.599609  10837.500000   
2  2003-01-06  10837.500000  10837.500000  10271.900391  10357.299805   
3  2003-01-07  10357.299805  10357.299805   9665.400391   9752.900391   
4  2003-01-08   9752.900391  10161.200195   9690.700195  10161.200195   

      Adj Close       Volume  
0  10598.599609  178541100.0  
1  10837.500000  180109700.0  
2  10357.299805  117221300.0  
3   9752.900391  151279600.0  
4  10161.200195  154326700.0  
Tarih            0
Açılış           0
Yüksek           0
Düşük            0
Kapanış          0
Adj_Kapanış      0
Hacim            0
Haftanın Günü    0
Ay               0
Ay Başı          0
Ay Sonu          0
H-L              0
dtype: int64
        Tarih        Açılış        Yüksek         Düşük       Kapanış  \
0  2003-01-02  10369.900391  10642.299